# 导入第三方包

In [3]:
import os
import gc
import math

import pandas as pd
import numpy as np

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

# 读取数据

In [6]:
path = '/Users/sampras/Desktop/学习资料/数据/心跳分类/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'testA.csv')
train.head()

,id,heartbeat_signals,label
0,0,"0.9912297987616655,0.9435330436439665,0.764677...",0.0
1,1,"0.9714822034884503,0.9289687459588268,0.572932...",0.0
2,2,"1.0,0.9591487564065292,0.7013782792997189,0.23...",2.0
3,3,"0.9757952826275774,0.9340884687738161,0.659636...",0.0
4,4,"0.0,0.055816398940721094,0.26129357194994196,0...",2.0


In [7]:
test.head()

,id,heartbeat_signals
0,100000,"0.9915713654170097,1.0,0.6318163407681274,0.13..."
1,100001,"0.6075533139615096,0.5417083883163654,0.340694..."
2,100002,"0.9752726292239277,0.6710965234906665,0.686758..."
3,100003,"0.9956348033996116,0.9170249621481004,0.521096..."
4,100004,"1.0,0.8879490481178918,0.745564725322326,0.531..."


# 数据预处理

In [9]:
def reduce_mem_usage(df):
    """
    优化内存
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(
                        np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(
                        np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(
                        np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(
                        np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(
                        np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(
                        np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) /
                                        start_mem))

    return df

In [10]:
# 简单预处理
train_list = []

for items in train.values:
    train_list.append([items[0]] + [float(i)
                                    for i in items[1].split(',')] + [items[2]])

train = pd.DataFrame(np.array(train_list))
train.columns = ['id'
                 ] + ['s_' + str(i)
                      for i in range(len(train_list[0]) - 2)] + ['label']
train = reduce_mem_usage(train)

test_list = []
for items in test.values:
    test_list.append([items[0]] + [float(i) for i in items[1].split(',')])

test = pd.DataFrame(np.array(test_list))
test.columns = ['id'] + ['s_' + str(i) for i in range(len(test_list[0]) - 1)]
test = reduce_mem_usage(test)

Memory usage of dataframe is 157.93 MB
Memory usage after optimization is: 39.67 MB
Decreased by 74.9%
Memory usage of dataframe is 31.43 MB
Memory usage after optimization is: 7.90 MB
Decreased by 74.9%


In [11]:
train.head()

,id,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,...,s_196,s_197,s_198,s_199,s_200,s_201,s_202,s_203,s_204,label
0,0.0,0.991211,0.943359,0.764648,0.618652,0.379639,0.190796,0.040222,0.026001,0.031708,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.971680,0.929199,0.572754,0.178467,0.122986,0.132324,0.094421,0.089600,0.030487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,1.000000,0.958984,0.701172,0.231812,0.000000,0.080688,0.128418,0.187500,0.280762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3.0,0.975586,0.934082,0.659668,0.249878,0.237061,0.281494,0.249878,0.249878,0.241455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.000000,0.055817,0.261230,0.359863,0.433105,0.453613,0.499023,0.542969,0.616699,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [12]:
test.head()

,id,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,...,s_195,s_196,s_197,s_198,s_199,s_200,s_201,s_202,s_203,s_204
0,100000.0,0.991699,1.000000,0.631836,0.136230,0.041412,0.102722,0.120850,0.123413,0.107910,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,100001.0,0.607422,0.541504,0.340576,0.000000,0.090698,0.164917,0.195068,0.168823,0.198853,...,0.389893,0.386963,0.367188,0.364014,0.360596,0.357178,0.350586,0.350586,0.350586,0.36377
2,100002.0,0.975098,0.670898,0.686523,0.708496,0.718750,0.716797,0.720703,0.701660,0.596680,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,100003.0,0.995605,0.916992,0.520996,0.000000,0.221802,0.404053,0.490479,0.527344,0.518066,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,100004.0,1.000000,0.888184,0.745605,0.531738,0.380371,0.224609,0.091125,0.057648,0.003914,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


# 训练数据/测试数据准备

In [13]:
x_train = train.drop(['id', 'label'], axis=1)
y_train = train['label']
x_test = test.drop(['id'], axis=1)

# 模型训练

In [14]:
def abs_sum(y_pre, y_tru):
    y_pre = np.array(y_pre)
    y_tru = np.array(y_tru)
    loss = sum(sum(abs(y_pre - y_tru)))
    return loss

In [15]:
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2021
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    test = np.zeros((test_x.shape[0], 4))

    cv_scores = []
    onehot_encoder = OneHotEncoder(sparse=False)
    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print(
            '************************************ {} ************************************'
            .format(str(i + 1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[
            train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'multiclass',
                'num_class': 4,
                'num_leaves': 2**5,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.1,
                'seed': seed,
                'nthread': 28,
                'n_jobs': 24,
                'verbose': -1,
            }

            model = clf.train(params,
                              train_set=train_matrix,
                              valid_sets=valid_matrix,
                              num_boost_round=2000,
                              verbose_eval=100,
                              early_stopping_rounds=200)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x,
                                      num_iteration=model.best_iteration)

        val_y = np.array(val_y).reshape(-1, 1)
        val_y = onehot_encoder.fit_transform(val_y)
        print('预测的概率矩阵为：')
        print(test_pred)
        test += test_pred
        score = abs_sum(val_y, val_pred)
        cv_scores.append(score)
        print(cv_scores)
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    test = test / kf.n_splits

    return test

In [16]:
def lgb_model(x_train, y_train, x_test):
    lgb_test = cv_model(lgb, x_train, y_train, x_test, "lgb")
    return lgb_test

In [17]:
lgb_test = lgb_model(x_train, y_train, x_test)

************************************ 1 ************************************
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_logloss: 0.064112
[200]	valid_0's multi_logloss: 0.0459167
[300]	valid_0's multi_logloss: 0.0408373
[400]	valid_0's multi_logloss: 0.0392399
[500]	valid_0's multi_logloss: 0.0392175
[600]	valid_0's multi_logloss: 0.0405053
Early stopping, best iteration is:
[463]	valid_0's multi_logloss: 0.0389485
预测的概率矩阵为：
[[9.99961153e-01 3.79594757e-05 5.40269890e-07 3.47753326e-07]
 [4.99636350e-05 5.92518438e-04 9.99357510e-01 8.40463147e-09]
 [1.13534131e-06 6.17413877e-08 5.21828651e-07 9.99998281e-01]
 ...
 [1.22083700e-01 2.13622465e-04 8.77693686e-01 8.99180470e-06]
 [9.99980119e-01 1.98198246e-05 3.10446404e-08 2.99597008e-08]
 [9.94006248e-01 1.02828877e-03 3.81770253e-03 1.14776091e-03]]
[579.1476207255505]
************************************ 2 ************************************
Training until validation scores don't improve for 

In [18]:
lgb_test

array([[9.99982410e-01, 1.68417503e-05, 3.72291800e-07, 3.75572656e-07],
       [4.00291038e-05, 6.93522828e-04, 9.99266433e-01, 1.49925017e-08],
       [1.67786048e-06, 1.10086204e-07, 8.39361754e-07, 9.99997373e-01],
       ...,
       [3.93414363e-02, 2.58535867e-04, 9.60393965e-01, 6.06322760e-06],
       [9.99954264e-01, 4.56448269e-05, 5.20499488e-08, 3.93312894e-08],
       [9.49136319e-01, 5.35406119e-03, 4.03220016e-02, 5.18761790e-03]])

In [19]:
temp = pd.DataFrame(lgb_test)
temp

,0,1,2,3
0,0.999982,1.684175e-05,3.722918e-07,3.755727e-07
1,0.000040,6.935228e-04,9.992664e-01,1.499250e-08
2,0.000002,1.100862e-07,8.393618e-07,9.999974e-01
3,0.999973,1.603527e-05,1.078714e-05,3.316444e-08
4,0.999985,2.957121e-06,1.219124e-05,1.352210e-07
...,...,...,...,...
19995,0.998782,2.364951e-04,7.980751e-05,9.015345e-04
19996,0.999915,7.776196e-05,6.719279e-06,8.058959e-08
19997,0.039341,2.585359e-04,9.603940e-01,6.063228e-06
19998,0.999954,4.564483e-05,5.204995e-08,3.933129e-08


In [21]:
result = pd.read_csv(path+'sample_submit.csv')
result['label_0'] = temp[0]
result['label_1'] = temp[1]
result['label_2'] = temp[2]
result['label_3'] = temp[3]
result.to_csv(path+'submit.csv', index=False)